In [1]:
from torch import empty, zeros, cat
from tqdm.autonotebook import tqdm

from torchkge.data_structures import SmallKG
from torchkge.exceptions import NotYetEvaluatedError
from torchkge.sampling import PositionalNegativeSampler
from torchkge.utils import DataLoader
import pandas as pd
import torch
import torch
import os
from os.path import join
# from evaluation import TemporalLinkPredictionEvaluator

from torchkge.utils import datasets
import torch
from pandas import read_csv, concat
import json
from datetime import datetime
import pandas as pd

from torchkge.utils import datasets
import torch
from pandas import read_csv, concat
# from data_utils import TemporalKnowledgeGraph
from numpy.random import RandomState

from torch import tensor, bernoulli, randint, ones, rand, cat


In [2]:
import shutil

from os import environ, makedirs
from os.path import exists, expanduser, join
from torch.utils.data import Dataset
from torchkge.exceptions import SizeMismatchError, WrongArgumentsError, SanityError
from torchkge.utils.operations import get_dictionaries
from torch import cat, eq, int64, long, randperm, tensor, Tensor, zeros_like
from collections import defaultdict
from pandas import DataFrame
import torch
# from utils import get_temporal_dictionaries
from pandas import read_csv, concat

In [3]:
from os.path import join
import torch
from torch import cuda
from torch.optim import Adam
from torchkge.sampling import BernoulliNegativeSampler, UniformNegativeSampler
from tqdm.notebook import tqdm
import pandas as pd
import os


In [4]:
from torchkge.utils import MarginLoss, BinaryCrossEntropyLoss
import torch
import torch.distributed as dist
from abc import ABC, abstractmethod
import torch.distributed as dist
import torch.nn as nn

import numpy as np

In [5]:
from torch.nn import Module


from torchkge.utils.dissimilarities import l1_dissimilarity, l2_dissimilarity, \
    l1_torus_dissimilarity, l2_torus_dissimilarity, el2_torus_dissimilarity

In [6]:
from collections import defaultdict

from torch import tensor, bernoulli, randint, ones, rand, cat

from torchkge.exceptions import NotYetImplementedError
from torchkge.utils.data import DataLoader
from torchkge.utils.operations import get_bernoulli_probs


In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

## Utils

In [8]:

tem_dict = {
    '0y': 0, '1y': 1, '2y': 2, '3y': 3, '4y': 4, '5y': 5, '6y': 6, '7y': 7, '8y': 8, '9y': 9,
    '0m': 10, '1m': 11, '2m': 12, '3m': 13, '4m': 14, '5m': 15, '6m': 16, '7m': 17, '8m': 18, '9m': 19,
    '0d': 20, '1d': 21, '2d': 22, '3d': 23, '4d': 24, '5d': 25, '6d': 26, '7d': 27, '8d': 28, '9d': 29
}

class Params():
    def __init__(self):
        pass

def read_json_lines(file_name):
    lines = []
    with open(file_name) as file_in:
        for line in file_in:
            lines.append(json.loads(line))
    return lines

def get_expriment_tests(results):
    res = []
    for exp in results:
        train_params = exp['train_params']
        test_results = exp['test_results']
        train_params.update(test_results)
        res.append(train_params)
    return pd.DataFrame(res)


def write_json_lines(file_name,dict_data):
    json_string = json.dumps(dict_data)
    with open(file_name, 'a') as f:
        f.write(json_string+"\n")

def experiment_exists(all_exp_data_df, exp_data_dict):
    for k in exp_data_dict:
        all_exp_data_df = all_exp_data_df[all_exp_data_df[k]==exp_data_dict[k]]
        if all_exp_data_df.shape[0]==0:
            return False
    return True
def check_experiment(exp_file_name):
    res = read_json_lines(exp_file_name)
    res = get_expriment_tests(res)
    return res
    # experiment_exists(all_exp_data_df, exp_data_dict)
def transform_time_V2(years, months, days):
    all_data = []
    for year, month, day in zip(years, months, days):
        tem_id_list = []
        for j in range(len(year)):
            token = year[j:j+1]+'y'
            tem_id_list.append(tem_dict[token])
        # print(tem_id_list)
        # exit()

        for j in range(1):
            # print(month[1])
            # exit()
            token1 = month[0]+'m'
            tem_id_list.append(tem_dict[token1])
            token2 = month[0]+'m'
            tem_id_list.append(tem_dict[token2])


        for j in range(len(day)):
            token = day[j:j+1]+'d'
            tem_id_list.append(tem_dict[token])
            
        all_data.append(torch.tensor(tem_id_list))
    return all_data

def transform_time(raw_time):
    year, month, day = raw_time.split("-")
    tem_id_list = []
    for j in range(len(year)):
        token = year[j:j+1]+'y'
        tem_id_list.append(tem_dict[token])
    # print(tem_id_list)
    # exit()

    for j in range(1):
        # print(month[1])
        # exit()
        token1 = month[0]+'m'
        tem_id_list.append(tem_dict[token1])
        token2 = month[0]+'m'
        tem_id_list.append(tem_dict[token2])


    for j in range(len(day)):
        token = day[j:j+1]+'d'
        tem_id_list.append(tem_dict[token])
    return tem_id_list

def transform_time_v3(raw_time):
    date = list(map(float, raw_time.split("-")))
    return date

def transform_time_v4(raw_time):
    year, month, day = raw_time.split("-")
    year, month, day = int(year), int(month), int(day)
    return month + day

def transform_time_v5(raw_time):
    year, month, day = raw_time.split("-")
    return [int(year), int(month), int(day)]

def get_temporal_dictionaries(df, mode='simple'):

    tmp = list(set(df['start_time'].unique()).union(set(df['end_time'].unique())))
    if mode == 'simple_time':
        # return {timee: i for i, timee in enumerate(sorted(tmp))}
        return {timee: i for i, timee in enumerate(sorted([datetime.strptime(dt, "%Y-%m-%d") for dt in tmp]))}
    elif mode == 'simple':
        # return {timee: i for i, timee in enumerate(sorted(tmp))}
        return {timee: i for i, timee in enumerate(sorted([dt for dt in tmp]))}
    elif mode == 'seq':
        return {timee: torch.tensor(transform_time(timee)) for i, timee in enumerate(sorted(tmp))}
        # return {timee: torch.tensor(transform_time(timee)) for i, timee in enumerate(sorted([datetime.strptime(dt, "%Y-%m-%d") for dt in tmp]))}
    elif mode == 'ymd':
        return {timee: transform_time_v4(timee) for i, timee in enumerate(sorted(tmp))}
    elif mode == 'ymd_':
        return {timee: torch.tensor(transform_time_v5(timee)) for i, timee in enumerate(sorted(tmp))}


def cconv(a, b):
    return torch.fft.ifft(torch.fft.fft(a) * torch.fft.fft(b)).real


def ccorr(a, b):
    return torch.fft.ifft(torch.conj(torch.fft.fft(a)) * torch.fft.fft(b)).real


##  Data Utils

In [9]:
class TemporalKnowledgeGraph(Dataset):

    def __init__(self, df=None, time_mode = None,kg=None, ent2ix=None, rel2ix=None, time2ix = None,
                 dict_of_heads=None, dict_of_tails=None, dict_of_rels=None,
                 temp_dict_of_heads=None, temp_dict_of_tails=None, temp_dict_of_rels=None):

        if df is None:
            if kg is None:
                raise WrongArgumentsError("Please provide at least one "
                                          "argument of `df` and kg`")
            else:
                try:
                    assert (type(kg) == dict) & ('heads' in kg.keys()) & \
                           ('tails' in kg.keys()) & \
                           ('relations' in kg.keys())& \
                            ('start_time' in kg.keys())& \
                            ('end_time' in kg.keys())
                    
                except AssertionError:
                    raise WrongArgumentsError("Keys in the `kg` dict should "
                                              "contain `heads`, `tails`, "
                                              "`relations`.")
                try:
                    assert (rel2ix is not None) & (ent2ix is not None)
                except AssertionError:
                    raise WrongArgumentsError("Please provide the two "
                                              "dictionaries ent2ix and rel2ix "
                                              "if building from `kg`.")
        else:
            if kg is not None:
                raise WrongArgumentsError("`df` and kg` arguments should not "
                                          "both be provided.")

        if ent2ix is None:
            self.ent2ix = get_dictionaries(df, ent=True)
        else:
            self.ent2ix = ent2ix

        if rel2ix is None:
            self.rel2ix = get_dictionaries(df, ent=False)
        else:
            self.rel2ix = rel2ix
        
        if time_mode is not None:
            self.time_mode = time_mode
            
        if time2ix is None:
            self.time2ix = get_temporal_dictionaries(df, mode = self.time_mode)
        else:
            self.time2ix = time2ix
        
        self.n_ent = max(self.ent2ix.values()) + 1
        self.n_rel = max(self.rel2ix.values()) + 1
        time_val = list(self.time2ix.values())
        
        if isinstance(time_val[0], torch.Tensor):
            self.n_time = int(torch.cat(time_val).max()) + 1
        else: 
            self.n_time = max(time_val) + 1
            
#         print('self.n_time: ',self.n_time)
        if df is not None:
            # build kg from a pandas dataframe
            self.n_facts = len(df)
            self.head_idx = tensor(df['from'].map(self.ent2ix).values).long()
            self.tail_idx = tensor(df['to'].map(self.ent2ix).values).long()
            self.relations = tensor(df['rel'].map(self.rel2ix).values).long()
#             self.start_time = tensor(df['start_time'].map(self.rel2ix).values).long()
#             self.end_time = tensor(df['end_time'].map(self.rel2ix).values).long()
#             print(self.time2ix)
            self.start_time = list(df['start_time'].map(self.time2ix).values)
#             print(self.start_time)
#             print('-'*22)
#             print("type(self.start_time[0]): ",type(self.start_time[0]))
#             print('type(self.start_time): ',type(self.start_time))
#             if isinstance(self.start_time[0], torch.Tensor):
#                 print('llllll')
            if isinstance(self.start_time, list) & isinstance(self.start_time[0], torch.Tensor):
                self.start_time = torch.stack(self.start_time)
            else:
                self.start_time = torch.tensor(self.start_time)
            
            self.start_time = self.start_time.long()
            
            self.end_time = list(df['end_time'].map(self.time2ix).values)
            if isinstance(self.end_time, list) & isinstance(self.end_time[0], torch.Tensor):
                self.end_time = torch.stack(self.end_time)
            else:
                self.end_time = torch.tensor(self.end_time)   
            self.end_time = self.end_time.long()
            
        else:
            # build kg from another kg
            self.n_facts = kg['heads'].shape[0]
            self.head_idx = kg['heads']
            self.tail_idx = kg['tails']
            self.relations = kg['relations']
            self.start_time = kg['start_time']
            self.end_time = kg['end_time']

        if dict_of_heads is None or dict_of_tails is None or dict_of_rels is None:
            self.dict_of_heads = defaultdict(set)
            self.dict_of_tails = defaultdict(set)
            self.dict_of_rels = defaultdict(set)
            self.temp_dict_of_heads = defaultdict(set)
            self.temp_dict_of_tails = defaultdict(set)
            self.temp_dict_of_rels = defaultdict(set)
#             self.dict_of_start_time = defaultdict(set)
#             self.dict_of_end_time = defaultdict(set)
            self.evaluate_dicts()

        else:
            self.dict_of_heads = dict_of_heads
            self.dict_of_tails = dict_of_tails
            self.dict_of_rels = dict_of_rels
            self.temp_dict_of_heads = temp_dict_of_heads
            self.temp_dict_of_tails = temp_dict_of_tails
            self.temp_dict_of_rels = temp_dict_of_rels
#             self.dict_of_start_time = dict_of_start_time
#             self.dict_of_end_time = dict_of_end_time
        try:
            self.sanity_check()
        except AssertionError:
            raise SanityError("Please check the sanity of arguments.")

    def __len__(self):
        return self.n_facts

    def __getitem__(self, item):
        return (self.head_idx[item].item(),
                self.tail_idx[item].item(),
                self.relations[item].item())

    def sanity_check(self):
        assert (type(self.dict_of_heads) == defaultdict) & \
               (type(self.dict_of_tails) == defaultdict) & \
               (type(self.dict_of_rels) == defaultdict) & \
                (type(self.temp_dict_of_heads) == defaultdict) & \
               (type(self.temp_dict_of_tails) == defaultdict) & \
               (type(self.temp_dict_of_rels) == defaultdict)
        assert (type(self.ent2ix) == dict) & (type(self.rel2ix) == dict)
        assert (len(self.ent2ix) == self.n_ent) & \
               (len(self.rel2ix) == self.n_rel)
        assert (type(self.head_idx) == Tensor) & \
               (type(self.tail_idx) == Tensor) & \
               (type(self.relations) == Tensor)
        assert (self.head_idx.dtype == int64) & \
               (self.tail_idx.dtype == int64) & (self.relations.dtype == int64)
        assert (len(self.head_idx) == len(self.tail_idx) == len(self.relations))

    def split_kg(self, share=0.8, sizes=None, validation=False):
        if sizes is not None:
            try:
                if len(sizes) == 3:
                    try:
                        assert (sizes[0] + sizes[1] + sizes[2] == self.n_facts)
                    except AssertionError:
                        raise WrongArgumentsError('Sizes should sum to the '
                                                  'number of facts.')
                elif len(sizes) == 2:
                    try:
                        assert (sizes[0] + sizes[1] == self.n_facts)
                    except AssertionError:
                        raise WrongArgumentsError('Sizes should sum to the '
                                                  'number of facts.')
                else:
                    raise SizeMismatchError('Tuple `sizes` should be of '
                                            'length 2 or 3.')
            except AssertionError:
                raise SizeMismatchError('Tuple `sizes` should sum up to the '
                                        'number of facts in the knowledge '
                                        'graph.')
        else:
            assert share < 1

        if ((sizes is not None) and (len(sizes) == 3)) or \
                ((sizes is None) and validation):
            # return training, validation and a testing graphs

            if (sizes is None) and validation:
                mask_tr, mask_val, mask_te = self.get_mask(share,
                                                           validation=True)
            else:
                mask_tr = cat([tensor([1 for _ in range(sizes[0])]),
                               tensor([0 for _ in range(sizes[1] + sizes[2])])]).bool()
                mask_val = cat([tensor([0 for _ in range(sizes[0])]),
                                tensor([1 for _ in range(sizes[1])]),
                                tensor([0 for _ in range(sizes[2])])]).bool()
                mask_te = ~(mask_tr | mask_val)

            return (TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_tr],
                            'tails': self.tail_idx[mask_tr],
                            'relations': self.relations[mask_tr],
                           'start_time': self.start_time[mask_tr],
                           'end_time': self.end_time[mask_tr]},
                            ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                            dict_of_heads=self.dict_of_heads,
                            dict_of_tails=self.dict_of_tails,
                            dict_of_rels=self.dict_of_rels,
                            temp_dict_of_heads = self.temp_dict_of_heads,
                            temp_dict_of_tails = self.temp_dict_of_tails,
                            temp_dict_of_rels = self.temp_dict_of_rels
                            ),
                    TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_val],
                            'tails': self.tail_idx[mask_val],
                            'relations': self.relations[mask_val],
                           'start_time':  self.start_time[mask_val],
                           'end_time': self.end_time[mask_val]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels),
                    TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_te],
                            'tails': self.tail_idx[mask_te],
                            'relations': self.relations[mask_te],
                           'start_time': self.start_time[mask_te],
                           'end_time': self.end_time[mask_te]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels
                        ))
        else:
            # return training and testing graphs

            assert (((sizes is not None) and len(sizes) == 2) or
                    ((sizes is None) and not validation))
            if sizes is None:
                mask_tr, mask_te = self.get_mask(share, validation=False)
            else:
                mask_tr = cat([tensor([1 for _ in range(sizes[0])]),
                               tensor([0 for _ in range(sizes[1])])]).bool()
                mask_te = ~mask_tr
            return (TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_tr],
                            'tails': self.tail_idx[mask_tr],
                            'relations': self.relations[mask_tr],
                           'start_time': self.start_time[mask_tr],
                           'end_time': self.end_time[mask_tr]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels
                        ),
                    TemporalKnowledgeGraph(
                        kg={'heads': self.head_idx[mask_te],
                            'tails': self.tail_idx[mask_te],
                            'relations': self.relations[mask_te],
                           'start_time': self.start_time[mask_te],
                           'end_time': self.end_time[mask_te]},
                        ent2ix=self.ent2ix, rel2ix=self.rel2ix, time2ix = self.time2ix,
                        dict_of_heads=self.dict_of_heads,
                        dict_of_tails=self.dict_of_tails,
                        dict_of_rels=self.dict_of_rels,
                        temp_dict_of_heads = self.temp_dict_of_heads,
                        temp_dict_of_tails = self.temp_dict_of_tails,
                        temp_dict_of_rels = self.temp_dict_of_rels
                        ))

    def get_mask(self, share, validation=False):

        uniques_r, counts_r = self.relations.unique(return_counts=True)
        uniques_e, _ = cat((self.head_idx,
                            self.tail_idx)).unique(return_counts=True)

        mask = zeros_like(self.relations).bool()
        if validation:
            mask_val = zeros_like(self.relations).bool()

        # splitting relations among subsets
        for i, r in enumerate(uniques_r):
            rand = randperm(counts_r[i].item())

            # list of indices k such that relations[k] == r
            sub_mask = eq(self.relations, r).nonzero(as_tuple=False)[:, 0]

            assert len(sub_mask) == counts_r[i].item()

            if validation:
                train_size, val_size, test_size = self.get_sizes(counts_r[i].item(),
                                                                 share=share,
                                                                 validation=True)
                mask[sub_mask[rand[:train_size]]] = True
                mask_val[sub_mask[rand[train_size:train_size + val_size]]] = True

            else:
                train_size, test_size = self.get_sizes(counts_r[i].item(),
                                                       share=share,
                                                       validation=False)
                mask[sub_mask[rand[:train_size]]] = True

        # adding missing entities to the train set
        u = cat((self.head_idx[mask], self.tail_idx[mask])).unique()
        if len(u) < self.n_ent:
            missing_entities = tensor(list(set(uniques_e.tolist()) -
                                           set(u.tolist())), dtype=long)
            for e in missing_entities:
                sub_mask = ((self.head_idx == e) |
                            (self.tail_idx == e)).nonzero(as_tuple=False)[:, 0]
                rand = randperm(len(sub_mask))
                sizes = self.get_sizes(mask.shape[0],
                                       share=share,
                                       validation=validation)
                mask[sub_mask[rand[:sizes[0]]]] = True
                if validation:
                    mask_val[sub_mask[rand[:sizes[0]]]] = False

        if validation:
            assert not (mask & mask_val).any().item()
            return mask, mask_val, ~(mask | mask_val)
        else:
            return mask, ~mask

    @staticmethod
    def get_sizes(count, share, validation=False):
        if count == 1:
            if validation:
                return 1, 0, 0
            else:
                return 1, 0
        if count == 2:
            if validation:
                return 1, 1, 0
            else:
                return 1, 1

        n_train = int(count * share)
        assert n_train < count
        if n_train == 0:
            n_train += 1

        if not validation:
            return n_train, count - n_train
        else:
            if count - n_train == 1:
                n_train -= 1
                return n_train, 1, 1
            else:
                n_val = int(int(count - n_train) / 2)
                return n_train, n_val, count - n_train - n_val

    def evaluate_dicts(self):
        for i in range(self.n_facts):
            self.dict_of_heads[(self.tail_idx[i].item(),
                                self.relations[i].item())].add(self.head_idx[i].item())
            self.dict_of_tails[(self.head_idx[i].item(),
                                self.relations[i].item())].add(self.tail_idx[i].item())
            self.dict_of_rels[(self.head_idx[i].item(),
                               self.tail_idx[i].item())].add(self.relations[i].item())
            
            self.temp_dict_of_heads[(self.tail_idx[i].item(),
                                    self.relations[i].item(),
                                     self.start_time[i].item(),
                                     self.end_time[i].item())].add(self.head_idx[i].item())
            self.temp_dict_of_tails[(self.head_idx[i].item(),
                                self.relations[i].item(),
                                     self.start_time[i].item(),
                                     self.end_time[i].item())].add(self.tail_idx[i].item())
            self.temp_dict_of_rels[(self.head_idx[i].item(),
                                   self.tail_idx[i].item(),
                                    self.start_time[i].item(),
                                     self.end_time[i].item())].add(self.relations[i].item())

    def get_df(self):
        ix2ent = {v: k for k, v in self.ent2ix.items()}
        ix2rel = {v: k for k, v in self.rel2ix.items()}

        df = DataFrame(cat((self.head_idx.view(1, -1),
                            self.tail_idx.view(1, -1),
                            self.relations.view(1, -1))).transpose(0, 1).numpy(),
                       columns=['from', 'to', 'rel'])

        df['from'] = df['from'].apply(lambda x: ix2ent[x])
        df['to'] = df['to'].apply(lambda x: ix2ent[x])
        df['rel'] = df['rel'].apply(lambda x: ix2rel[x])

        return df


class SmallKG(Dataset):
    def __init__(self, heads, tails, relations):
        assert heads.shape == tails.shape == relations.shape
        self.head_idx = heads
        self.tail_idx = tails
        self.relations = relations
        self.length = heads.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, item):
        return self.head_idx[item].item(), self.tail_idx[item].item(), self.relations[item].item()



def get_data_home(data_home=None):
    if data_home is None:
        data_home = environ.get('TORCHKGE_DATA',
                                join('~', 'torchkge_data'))
    data_home = expanduser(data_home)
    if not exists(data_home):
        makedirs(data_home)
    return data_home


def clear_data_home(data_home=None):
    data_home = get_data_home(data_home)
    shutil.rmtree(data_home)


def get_n_batches(n, b_size):
    n_batch = n // b_size
    if n % b_size > 0:
        n_batch += 1
    return n_batch


class TempDataLoader:
    """This class is inspired from :class:`torch.utils.dataloader.DataLoader`.
    It is however way simpler.

    """
    def __init__(self, kg, batch_size, use_cuda=None):

        self.h = kg.head_idx
        self.t = kg.tail_idx
        self.r = kg.relations
        self.start_time = kg.start_time
        self.end_time = kg.end_time

        self.use_cuda = use_cuda
        self.batch_size = batch_size

        if use_cuda is not None and use_cuda == 'all':
            self.h = self.h.cuda()
            self.t = self.t.cuda()
            self.r = self.r.cuda()
            self.start_time = self.start_time.cuda()
            self.end_time = self.end_time.cuda()
            
    def __len__(self):
        return get_n_batches(len(self.h), self.batch_size)

    def __iter__(self):
        return _TempDataLoaderIter(self)


class _TempDataLoaderIter:
    def __init__(self, loader):
        self.h = loader.h
        self.t = loader.t
        self.r = loader.r
        self.start_time = loader.start_time
        self.end_time = loader.end_time
        
        self.use_cuda = loader.use_cuda
        self.batch_size = loader.batch_size

        self.n_batches = get_n_batches(len(self.h), self.batch_size)
        self.current_batch = 0

    def __next__(self):
        if self.current_batch == self.n_batches:
            raise StopIteration
        else:
            i = self.current_batch
            self.current_batch += 1

            tmp_h = self.h[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_t = self.t[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_r = self.r[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_start_time = self.start_time[i * self.batch_size: (i + 1) * self.batch_size]
            tmp_end_time = self.end_time[i * self.batch_size: (i + 1) * self.batch_size]
            
            if self.use_cuda is not None and self.use_cuda == 'batch':
                return tmp_h.cuda(), tmp_t.cuda(), tmp_r.cuda(), tmp_start_time.cuda(), tmp_end_time.cuda()
            else:
                return tmp_h, tmp_t, tmp_r, tmp_start_time, tmp_end_time

    def __iter__(self):
        return self


def load_icews14(data_home=None):
    data_path = data_home + '/icews14'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode='seq')

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))


## Quat Utils

In [10]:
def quaternion_mul_with_unit_norm(*, Q_1, Q_2):
    a_h, b_h, c_h, d_h = Q_1  # = {a_h + b_h i + c_h j + d_h k : a_r, b_r, c_r, d_r \in R^k}
    a_r, b_r, c_r, d_r = Q_2  # = {a_r + b_r i + c_r j + d_r k : a_r, b_r, c_r, d_r \in R^k}

    # Normalize the relation to eliminate the scaling effect
    denominator = torch.sqrt(a_r ** 2 + b_r ** 2 + c_r ** 2 + d_r ** 2)
    p = a_r / denominator
    q = b_r / denominator
    u = c_r / denominator
    v = d_r / denominator
    #  Q'=E Hamilton product R
    r_val = a_h * p - b_h * q - c_h * u - d_h * v
    i_val = a_h * q + b_h * p + c_h * v - d_h * u
    j_val = a_h * u - b_h * v + c_h * p + d_h * q
    k_val = a_h * v + b_h * u - c_h * q + d_h * p
    return r_val, i_val, j_val, k_val


def quaternion_mul(*, Q_1, Q_2):
    a_h, b_h, c_h, d_h = Q_1  # = {a_h + b_h i + c_h j + d_h k : a_r, b_r, c_r, d_r \in R^k}
    a_r, b_r, c_r, d_r = Q_2  # = {a_r + b_r i + c_r j + d_r k : a_r, b_r, c_r, d_r \in R^k}
    r_val = a_h * a_r - b_h * b_r - c_h * c_r - d_h * d_r
    i_val = a_h * b_r + b_h * a_r + c_h * d_r - d_h * c_r
    j_val = a_h * c_r - b_h * d_r + c_h * a_r + d_h * b_r
    k_val = a_h * d_r + b_h * c_r - c_h * b_r + d_h * a_r
    return r_val, i_val, j_val, k_val

## Datasets

In [11]:
def get_data(data_name, time_mode):
    if data_name == 'icews14':
        return load_icews14(data_home='data', time_mode = time_mode)
    elif data_name == 'gdelt':
        return load_gdelt(data_home='data', time_mode = time_mode)
    elif data_name == 'icews15':
        return load_icews15(data_home='data', time_mode = time_mode)
    elif data_name == 'yago11k':
        return load_yago11k(data_home='data', time_mode = time_mode)
    elif data_name == 'wikidata12k':
        return load_wikidata12k(data_home='data', time_mode = time_mode)
    else:
        datas = ['icews14']
        
        print('Choose One of the Following Datasets: ',datas)

def load_icews14(data_home=None, time_mode=None):
    data_path = data_home + '/icews14'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_gdelt(data_home=None, time_mode=None):
    data_path = data_home + '/gdelt'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time'])
    df1['end_time'] = '1111-11-11'
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time'])
    df2['end_time'] = '1111-11-11'
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time'])
    df3['end_time'] = '1111-11-11'
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_icews15(data_home=None, time_mode=None):
    data_path = data_home + '/icews05-15'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_yago11k(data_home=None, time_mode=None):
    data_path = data_home + '/YAGO11k'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    df['start_time'] = df['start_time'].str.replace('##','01')
    df['end_time'] = df['end_time'].str.replace('##','01')
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

def load_wikidata12k(data_home=None, time_mode=None):
    data_path = data_home + '/WIKIDATA12k'

    df1 = read_csv(data_path + '/train.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df2 = read_csv(data_path + '/valid.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df3 = read_csv(data_path + '/test.txt',
                   sep='\t', header=None, names=['from', 'rel', 'to', 'start_time', 'end_time'])
    df = concat([df1, df2, df3])
    df['start_time'] = df['start_time'].str.replace('##','01')
    df['end_time'] = df['end_time'].str.replace('##','01')
    kg = TemporalKnowledgeGraph(df, time_mode = time_mode)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))


## Eval Utils

In [12]:
def get_true_targets(dictionary, key1, key2, key3, key4, true_idx, i):
    try:
        if key4 is not None:
            true_targets = dictionary[key1[i].item(), key2[i].item(), key3[i].item(), key4[i].item()].copy()
        else:
            true_targets = dictionary[key1[i].item(), key2[i].item(), key3[i].item()].copy()
            
        if true_idx is not None:
            true_targets.remove(true_idx[i].item())
            if len(true_targets) > 0:
                return tensor(list(true_targets)).long()
            else:
                return None
        else:
            return tensor(list(true_targets)).long()
    except KeyError:
        return None

In [13]:
# filter_scores(scores, self.kg.dict_of_tails, h_idx, r_idx, t_idx, start_time, end_time)
def filter_scores(scores, dictionary, key1, key2, key3, key4, true_idx):
    # filter out the true negative samples by assigning - inf score.
    b_size = scores.shape[0]
    filt_scores = scores.clone()

    for i in range(b_size):
        true_targets = get_true_targets(dictionary, key1, key2, key3, key4, true_idx, i)
        if true_targets is None:
            continue
        filt_scores[i][true_targets] = - float('Inf')

    return filt_scores


In [14]:
def get_rank(data, true, low_values=False):
    true_data = data.gather(1, true.long().view(-1, 1))

    if low_values:
        return (data <= true_data).sum(dim=1)
    else:
        return (data >= true_data).sum(dim=1)

## Evaluation

In [15]:
class TemporalLinkPredictionEvaluator(object):

    def __init__(self, model, knowledge_graph):
        self.model = model
        self.kg = knowledge_graph

        self.rank_true_heads = empty(size=(knowledge_graph.n_facts,)).long()
        self.rank_true_tails = empty(size=(knowledge_graph.n_facts,)).long()
        self.filt_rank_true_heads = empty(size=(knowledge_graph.n_facts,)).long()
        self.filt_rank_true_tails = empty(size=(knowledge_graph.n_facts,)).long()

        self.evaluated = False

    def evaluate(self, b_size, verbose=True):
        use_cuda = next(self.model.parameters()).is_cuda

        if use_cuda:
            dataloader = TempDataLoader(self.kg, batch_size=b_size, use_cuda='batch')
            self.rank_true_heads = self.rank_true_heads.cuda()
            self.rank_true_tails = self.rank_true_tails.cuda()
            self.filt_rank_true_heads = self.filt_rank_true_heads.cuda()
            self.filt_rank_true_tails = self.filt_rank_true_tails.cuda()
        else:
            dataloader = TempDataLoader(self.kg, batch_size=b_size)

        for i, batch in tqdm(enumerate(dataloader), total=len(dataloader),
                             unit='batch', disable=(not verbose),
                             desc='Link prediction evaluation'):
            h_idx, t_idx, r_idx, start_time, end_time = batch[0], batch[1], batch[2], batch[3], batch[4]
                        
            h_emb, t_emb, r_emb, candidates = self.model.inference_prepare_candidates(h_idx, t_idx, r_idx, start_time, end_time, entities=True)

            scores = self.model.inference_scoring_function(h_emb, candidates, r_emb, start_time, end_time)
            filt_scores = filter_scores(scores, self.kg.temp_dict_of_tails, h_idx, r_idx, start_time, end_time, t_idx)
            self.rank_true_tails[i * b_size: (i + 1) * b_size] = get_rank(scores, t_idx).detach()
            self.filt_rank_true_tails[i * b_size: (i + 1) * b_size] = get_rank(filt_scores, t_idx).detach()

            scores = self.model.inference_scoring_function(candidates, t_emb, r_emb, start_time, end_time)
            filt_scores = filter_scores(scores, self.kg.temp_dict_of_heads, t_idx, r_idx, start_time, end_time, h_idx)
            self.rank_true_heads[i * b_size: (i + 1) * b_size] = get_rank(scores, h_idx).detach()
            self.filt_rank_true_heads[i * b_size: (i + 1) * b_size] = get_rank(filt_scores, h_idx).detach()

        self.evaluated = True

        if use_cuda:
            self.rank_true_heads = self.rank_true_heads.cpu()
            self.rank_true_tails = self.rank_true_tails.cpu()
            self.filt_rank_true_heads = self.filt_rank_true_heads.cpu()
            self.filt_rank_true_tails = self.filt_rank_true_tails.cpu()

    def mean_rank(self):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        sum_ = (self.rank_true_heads.float().mean() +
                self.rank_true_tails.float().mean()).item()
        filt_sum = (self.filt_rank_true_heads.float().mean() +
                    self.filt_rank_true_tails.float().mean()).item()
        # return sum_ / 2, filt_sum / 2
        return {'mr':sum_ / 2, 'filt_mr':filt_sum / 2}

    def hit_at_k_heads(self, k=10):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        head_hit = (self.rank_true_heads <= k).float().mean()
        filt_head_hit = (self.filt_rank_true_heads <= k).float().mean()

        # return head_hit.item(), filt_head_hit.item()
        return {'head_hit_'+str(k):head_hit.item(), 'filt_head_hit_'+str(k): filt_head_hit.item()}

    def hit_at_k_tails(self, k=10):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        tail_hit = (self.rank_true_tails <= k).float().mean()
        filt_tail_hit = (self.filt_rank_true_tails <= k).float().mean()

        # return tail_hit.item(), filt_tail_hit.item()
        return {'tail_hit_'+str(k):tail_hit.item(), 'filt_tail_hit_'+str(k):filt_tail_hit.item()}

    def hit_at_k(self, k=10):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')

        head_hit = self.hit_at_k_heads(k=k)
        head_hit, filt_head_hit = head_hit['head_hit_'+str(k)], head_hit['filt_head_hit_'+str(k)]

        tail_hit = self.hit_at_k_tails(k=k)
        tail_hit, filt_tail_hit = tail_hit['tail_hit_'+str(k)], tail_hit['filt_tail_hit_'+str(k)]

        # return (head_hit + tail_hit) / 2, (filt_head_hit + filt_tail_hit) / 2
        return {'hit_'+str(k): (head_hit + tail_hit) / 2, 'filt_hit_'+str(k): (filt_head_hit + filt_tail_hit) / 2}

    def mrr(self):
        if not self.evaluated:
            raise NotYetEvaluatedError('Evaluator not evaluated call '
                                       'LinkPredictionEvaluator.evaluate')
        res = {}
        head_mrr = (self.rank_true_heads.float()**(-1)).mean()
        res['head_mrr'] = head_mrr.item()
        tail_mrr = (self.rank_true_tails.float()**(-1)).mean()
        res['tail_mrr'] = tail_mrr.item()
        filt_head_mrr = (self.filt_rank_true_heads.float()**(-1)).mean()
        res['filt_head_mrr'] = filt_head_mrr.item()
        filt_tail_mrr = (self.filt_rank_true_tails.float()**(-1)).mean()    
        res['filt_tail_mrr'] = filt_tail_mrr.item()
        res['mrr'] = (head_mrr + tail_mrr).item() / 2
        res['filt_mrr'] = (filt_head_mrr + filt_tail_mrr).item() / 2
        return res

    def get_results(self, k=None):
        res = {}
        if k is None:
            k = 10

        for i in range(1, k + 1):
            hits_res = self.hit_at_k(k=i)
            res.update(hits_res)

        mr_res = self.mean_rank()
        res.update(mr_res)
        mrr_res = self.mrr()
        res.update(mrr_res)
        return res

    def print_results(self, k=None, n_digits=3):
        if k is None:
            k = 10

        if k is not None and type(k) == int:
            print('Hit@{} : {} \t\t Filt. Hit@{} : {}'.format(
                k, round(self.hit_at_k(k=k)[0], n_digits),
                k, round(self.hit_at_k(k=k)[1], n_digits)))
        if k is not None and type(k) == list:
            for i in k:
                print('Hit@{} : {} \t\t Filt. Hit@{} : {}'.format(
                    i, round(self.hit_at_k(k=i)[0], n_digits),
                    i, round(self.hit_at_k(k=i)[1], n_digits)))

        print('Mean Rank : {} \t Filt. Mean Rank : {}'.format(
            int(self.mean_rank()[0]), int(self.mean_rank()[1])))
        print('MRR : {} \t\t Filt. MRR : {}'.format(
            round(self.mrr()[0], n_digits), round(self.mrr()[1], n_digits)))


## Losses

In [16]:
def get_loss(loss_name, args):
    if loss_name == 'MarginLoss':
        return MarginLoss(args.margin)
    elif loss_name == 'BinaryCrossEntropyLoss':
        return BinaryCrossEntropyLoss()

## Negative Sampling

## Interfaces

In [17]:
class TempModel(Module):
    def __init__(self, n_entities, n_relations):
        super().__init__()
        self.n_ent = n_entities
        self.n_rel = n_relations

    def forward(self, heads, tails, relations, start_time, end_time, negative_heads, negative_tails, negative_relations=None):
        pos = self.scoring_function(heads, tails, relations, start_time, end_time)

        if negative_relations is None:
            negative_relations = relations

        if negative_heads.shape[0] > negative_relations.shape[0]:
            # in that case, several negative samples are sampled from each fact
            n_neg = int(negative_heads.shape[0] / negative_relations.shape[0])
            # print('pos.shape: ', pos.shape)
            pos = pos.repeat(n_neg)
            neg = self.scoring_function(negative_heads,
                                        negative_tails,
                                        negative_relations.repeat(n_neg),
                                        start_time.repeat(n_neg, 1) if start_time.dim() ==2 & start_time.shape[-1]>1 else start_time.repeat(n_neg),
                                        end_time.repeat(n_neg, 1) if end_time.dim() ==2 & end_time.shape[-1]>1 else end_time.repeat(n_neg))
        else:
            neg = self.scoring_function(negative_heads,
                                        negative_tails,
                                        negative_relations, start_time, end_time)

        return pos, neg

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        raise NotImplementedError

    def normalize_parameters(self):
        raise NotImplementedError

    def get_embeddings(self):
        raise NotImplementedError

    def inference_scoring_function(self, h, t, r, time):
        raise NotImplementedError

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, entities=True):
        raise NotImplementedError


class TempTranslationModel(TempModel):
    def __init__(self, n_entities, n_relations, dissimilarity_type):
        super().__init__(n_entities, n_relations)

        assert dissimilarity_type in ['L1', 'L2', 'torus_L1', 'torus_L2',
                                      'torus_eL2']

        if dissimilarity_type == 'L1':
            self.dissimilarity = l1_dissimilarity
        elif dissimilarity_type == 'L2':
            self.dissimilarity = l2_dissimilarity
        elif dissimilarity_type == 'torus_L1':
            self.dissimilarity = l1_torus_dissimilarity
        elif dissimilarity_type == 'torus_L2':
            self.dissimilarity = l2_torus_dissimilarity
        else:
            self.dissimilarity = el2_torus_dissimilarity

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def normalize_parameters(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def get_embeddings(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, entities=True):
        raise NotImplementedError

    def inference_scoring_function(self, proj_h, proj_t, r, start_time, end_time):
        b_size = proj_h.shape[0]

        if len(r.shape) == 2:
            if len(proj_t.shape) == 3:
                assert (len(proj_h.shape) == 2)
                # this is the tail completion case in link prediction
                hr = (proj_h + r).view(b_size, 1, r.shape[1])
                return - self.dissimilarity(hr, proj_t)
            else:
                assert (len(proj_h.shape) == 3) & (len(proj_t.shape) == 2)
                # this is the head completion case in link prediction
                r_ = r.view(b_size, 1, r.shape[1])
                t_ = proj_t.view(b_size, 1, r.shape[1])
                return - self.dissimilarity(proj_h + r_, t_)
        elif len(r.shape) == 3:
            # this is the relation prediction case
            # Two cases possible:
            # * proj_ent.shape == (b_size, self.n_rel, self.emb_dim) -> projection depending on relations
            # * proj_ent.shape == (b_size, self.emb_dim) -> no projection
            proj_h = proj_h.view(b_size, -1, self.emb_dim)
            proj_t = proj_t.view(b_size, -1, self.emb_dim)
            return - self.dissimilarity(proj_h + r, proj_t)


class TempBilinearModel(TempModel):

    def __init__(self, emb_dim, n_entities, n_relations):
        super().__init__(n_entities, n_relations)
        self.emb_dim = emb_dim

    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def normalize_parameters(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def get_embeddings(self):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def inference_scoring_function(self, h, t, r, start_time, end_time):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx, start_time, end_time, entities=True):
        """See torchkge.models.interfaces.Models.
        """
        raise NotImplementedError

## Models

In [18]:
def scale_reg(h,t,r):
    s_a, x_a, y_a, z_a = torch.chunk(h, 4, dim =-1)
    s_c, x_c, y_c, z_c = torch.chunk(t, 4, dim =-1)
    s_b, x_b, y_b, z_b = torch.chunk(t, 4, dim =-1)
    regul = (torch.mean( torch.abs(s_a) ** 2)
                 + torch.mean( torch.abs(x_a) ** 2)
                 + torch.mean( torch.abs(y_a) ** 2)
                 + torch.mean( torch.abs(z_a) ** 2)
                 + torch.mean( torch.abs(s_c) ** 2)
                 + torch.mean( torch.abs(x_c) ** 2)
                 + torch.mean( torch.abs(y_c) ** 2)
                 + torch.mean( torch.abs(z_c) ** 2)
                 )
    regul2 =  (torch.mean( torch.abs(s_b) ** 2 )
             + torch.mean( torch.abs(x_b) ** 2 )
             + torch.mean( torch.abs(y_b) ** 2 )
             + torch.mean( torch.abs(z_b) ** 2 ))
    return 0.2 * (regul + regul2)


In [19]:
def quaternion_init(in_features, out_features, criterion='he'):
    fan_in = in_features
    fan_out = out_features

    if criterion == 'glorot':
        s = 1. / np.sqrt(2 * (fan_in + fan_out))
    elif criterion == 'he':
        s = 1. / np.sqrt(2 * fan_in)
    else:
        raise ValueError('Invalid criterion: ', criterion)
    rng = RandomState(123)

    # Generating randoms and purely imaginary quaternions :
    kernel_shape = (in_features, out_features)

    number_of_weights = np.prod(kernel_shape)
    v_i = np.random.uniform(0.0, 1.0, number_of_weights)
    v_j = np.random.uniform(0.0, 1.0, number_of_weights)
    v_k = np.random.uniform(0.0, 1.0, number_of_weights)

    # Purely imaginary quaternions unitary
    for i in range(0, number_of_weights):
        norm = np.sqrt(v_i[i] ** 2 + v_j[i] ** 2 + v_k[i] ** 2) + 0.0001
        v_i[i] /= norm
        v_j[i] /= norm
        v_k[i] /= norm
    v_i = v_i.reshape(kernel_shape)
    v_j = v_j.reshape(kernel_shape)
    v_k = v_k.reshape(kernel_shape)

    modulus = rng.uniform(low=-s, high=s, size=kernel_shape)
    phase = rng.uniform(low=-np.pi, high=np.pi, size=kernel_shape)

    weight_r = modulus * np.cos(phase)
    weight_i = modulus * v_i * np.sin(phase)
    weight_j = modulus * v_j * np.sin(phase)
    weight_k = modulus * v_k * np.sin(phase)

    return (weight_r, weight_i, weight_j, weight_k)

In [20]:
def oct_init(in_features, out_features, criterion='he'):
    fan_in = in_features
    fan_out = out_features

    if criterion == 'glorot':
        s = 1. / np.sqrt(2 * (fan_in + fan_out))
    elif criterion == 'he':
        s = 1. / np.sqrt(2 * fan_in)
    else:
        raise ValueError('Invalid criterion: ', criterion)
    rng = RandomState(123)

    # Generating randoms and purely imaginary quaternions :
    kernel_shape = (in_features, out_features)

    number_of_weights = np.prod(kernel_shape)
    v_i = np.random.uniform(0.0, 1.0, number_of_weights)
    v_j = np.random.uniform(0.0, 1.0, number_of_weights)
    v_k = np.random.uniform(0.0, 1.0, number_of_weights)
    v_l = np.random.uniform(0.0, 1.0, number_of_weights)
    v_m = np.random.uniform(0.0, 1.0, number_of_weights)
    v_n = np.random.uniform(0.0, 1.0, number_of_weights)
    v_o = np.random.uniform(0.0, 1.0, number_of_weights)
    # Purely imaginary quaternions unitary
    for i in range(0, number_of_weights):
        norm = np.sqrt(v_i[i] ** 2 + v_j[i] ** 2 + v_k[i] ** 2) + 0.0001
        v_i[i] /= norm
        v_j[i] /= norm
        v_k[i] /= norm
        v_l[i] /= norm
        v_m[i] /= norm
        v_n[i] /= norm
        v_o[i] /= norm
    v_i = v_i.reshape(kernel_shape)
    v_j = v_j.reshape(kernel_shape)
    v_k = v_k.reshape(kernel_shape)
    v_l = v_l.reshape(kernel_shape)
    v_m = v_m.reshape(kernel_shape)
    v_n = v_n.reshape(kernel_shape)
    v_o = v_o.reshape(kernel_shape)

    modulus = rng.uniform(low=-s, high=s, size=kernel_shape)
    phase = rng.uniform(low=-np.pi, high=np.pi, size=kernel_shape)

    weight_r = modulus * np.cos(phase)
    weight_i = modulus * v_i * np.sin(phase)
    weight_j = modulus * v_j * np.sin(phase)
    weight_k = modulus * v_k * np.sin(phase)
    weight_l = modulus * v_l * np.sin(phase)
    weight_m = modulus * v_m * np.sin(phase)
    weight_n = modulus * v_n * np.sin(phase)
    weight_o = modulus * v_o * np.sin(phase)
    return (weight_r, weight_i, weight_j, weight_k, weight_l, weight_m, weight_n, weight_o)

In [21]:
def quat_norm(q):
    s_b, x_b, y_b, z_b = torch.chunk(q, 4, dim=-1)
    denominator_b = torch.sqrt(s_b ** 2 + x_b ** 2 + y_b ** 2 + z_b ** 2)
    s_b = s_b / denominator_b
    x_b = x_b / denominator_b
    y_b = y_b / denominator_b
    z_b = z_b / denominator_b
    return torch.cat([s_b, x_b, y_b, z_b], dim=-1)

def hamilton_quat_prod(q1, q2):
    s_a, x_a, y_a, z_a = torch.chunk(q1, 4, dim=-1)
    s_b, x_b, y_b, z_b = torch.chunk(q2, 4, dim=-1)
    
    A = s_a * s_b - x_a * x_b - y_a * y_b - z_a * z_b
    B = s_a * x_b + s_b * x_a + y_a * z_b - y_b * z_a
    C = s_a * y_b + s_b * y_a + z_a * x_b - z_b * x_a
    D = s_a * z_b + s_b * z_a + x_a * y_b - x_b * y_a

    return torch.cat([A, B, C, D], dim=-1)

def quat_dot_prod(q1, q2):
    return torch.sum(q1* q2, -1)

def _onorm(q):
    r_1, r_2, r_3, r_4, r_5, r_6, r_7, r_8 = torch.chunk(q, 8, dim=-1)
    denominator = torch.sqrt(r_1 ** 2 + r_2 ** 2 + r_3 ** 2 + r_4 ** 2
                             + r_5 ** 2 + r_6 ** 2 + r_7 ** 2 + r_8 ** 2)
    r_1 = r_1 / denominator
    r_2 = r_2 / denominator
    r_3 = r_3 / denominator
    r_4 = r_4 / denominator
    r_5 = r_5 / denominator
    r_6 = r_6 / denominator
    r_7 = r_7 / denominator
    r_8 = r_8 / denominator

    return  torch.cat([r_1, r_2, r_3, r_4, r_5, r_6, r_7, r_8], dim=-1)

def _qmult(s_a, x_a, y_a, z_a, s_b, x_b, y_b, z_b):
    A = s_a * s_b - x_a * x_b - y_a * y_b - z_a * z_b
    B = s_a * x_b + s_b * x_a + y_a * z_b - y_b * z_a
    C = s_a * y_b + s_b * y_a + z_a * x_b - z_b * x_a
    D = s_a * z_b + s_b * z_a + x_a * y_b - x_b * y_a
    return A, B, C, D

def _qstar(a, b, c, d):
    return a, -b, -c, -d

def _omult(q1, q2):
    a_1, a_2, a_3, a_4, b_1, b_2, b_3, b_4 = torch.chunk(q1, 8, dim=-1)
    c_1, c_2, c_3, c_4, d_1, d_2, d_3, d_4 = torch.chunk(q2, 8, dim=-1)
    d_1_star, d_2_star, d_3_star, d_4_star = _qstar(d_1, d_2, d_3, d_4)
    c_1_star, c_2_star, c_3_star, c_4_star = _qstar(c_1, c_2, c_3, c_4)

    o_1, o_2, o_3, o_4 = _qmult(a_1, a_2, a_3, a_4, c_1, c_2, c_3, c_4 )
    o_1s, o_2s, o_3s, o_4s = _qmult(d_1_star, d_2_star, d_3_star, d_4_star,  b_1, b_2, b_3, b_4)

    o_5, o_6, o_7, o_8 = _qmult(d_1, d_2, d_3, d_4, a_1, a_2, a_3, a_4 )
    o_5s, o_6s, o_7s, o_8s = _qmult( b_1, b_2, b_3, b_4, c_1_star, c_2_star, c_3_star, c_4_star)

    return  torch.cat([o_1 - o_1s, o_2 - o_2s, o_3 - o_3s, o_4 - o_4s,
            o_5 + o_5s, o_6 + o_6s, o_7 + o_7s, o_8 + o_8s], dim=-1)

In [22]:
class TDYCE_Quat(TempBilinearModel):
    def __init__(self, args):
        super().__init__(args.emb_dim, args.n_entities, args.n_relations)
        self.emb_dim = args.emb_dim
        self.tem_total = args.tem_total
        self.quat_emb_dim = self.emb_dim * 4
        self.ent_emb = nn.Embedding(self.n_ent, self.quat_emb_dim)
        self.rel_emb = nn.Embedding(self.n_rel, self.quat_emb_dim)
        self.rel_transfer = nn.Embedding(self.n_rel, self.quat_emb_dim)
        self.time_transfer = nn.Embedding(self.tem_total, self.quat_emb_dim)
        self.init_weights()
        
    def init_weights(self):
        r, i, j, k = quaternion_init(self.n_ent, self.emb_dim)
        r, i, j, k = torch.from_numpy(r), torch.from_numpy(i), torch.from_numpy(j), torch.from_numpy(k)
        vec1 = torch.cat([r, i, j, k], dim=1)
        self.ent_emb.weight.data = vec1.type_as(self.ent_emb.weight.data)

        s, x, y, z = quaternion_init(self.n_rel, self.emb_dim)
        s, x, y, z = torch.from_numpy(s), torch.from_numpy(x), torch.from_numpy(y), torch.from_numpy(z)
        vec2 = torch.cat([s, x, y, z], dim=1)
        self.rel_emb.data = vec2.type_as(self.rel_emb.weight.data)

        s, x, y, z = quaternion_init(self.n_rel, self.emb_dim)
        s, x, y, z = torch.from_numpy(s), torch.from_numpy(x), torch.from_numpy(y), torch.from_numpy(z)
        vec2 = torch.cat([s, x, y, z], dim=1)
        self.rel_transfer.data = vec2.type_as(self.rel_transfer.weight.data)

        r, i, j, k = quaternion_init(self.n_ent, self.emb_dim)
        r, i, j, k = torch.from_numpy(r), torch.from_numpy(i), torch.from_numpy(j), torch.from_numpy(k)
        vec1 = torch.cat([r, i, j, k], dim=1)
        self.time_transfer.weight.data = vec1.type_as(self.time_transfer.weight.data)

    def _calc(self, h, r):
        return hamilton_quat_prod(h, quat_norm(r))

    def eval_calc(self, h, r):
        r = quat_norm(r)
        if (len(h.shape) == 3) & (len(r.shape) == 2):
            r = r.view(r.shape[0], 1, r.shape[1])
        elif (len(r.shape) == 3) & (len(h.shape) == 2):
            r = r.view(r.shape[0], 1, r.shape[1])
        return hamilton_quat_prod(h, r)

    def _transfer(self, x, x_transfer, r_transfer):
        ent_transfer = self._calc(x, x_transfer)
        ent_rel_transfer = self._calc(ent_transfer, r_transfer)

        return ent_rel_transfer

    def eval_transfer(self, x, x_transfer, r_transfer):
        ent_transfer = self.eval_calc(x, x_transfer)
        # print('ent_transfer.shape: ',ent_transfer.shape)
        # print('r_transfer.shape: ',r_transfer.shape)
        ent_rel_transfer = self.eval_calc(ent_transfer, r_transfer)

        return ent_rel_transfer
    
    
    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        h = self.ent_emb(h_idx)
        r = self.rel_emb(r_idx)
        t = self.ent_emb(t_idx)
        # (h, r, t) transfer vector
        time_transfer = self.time_transfer(start_time_idx)
        r_transfer = self.rel_transfer(r_idx)

        h1 = self._transfer(h, time_transfer, r_transfer)
        t1 = self._transfer(t, time_transfer, r_transfer)
        # multiplication as QuatE
        hr = self._calc(h1, r)
        # Inner product as QuatE
        score = quat_dot_prod(hr, t1)
        return score

    def get_embeddings(self):
        self.normalize_parameters()
        return self.ent_emb.weight.data, self.rel_emb.weight.data

    def inference_scoring_function(self, h, t, r, start_time, end_time):
        b_size = h.shape[0]
        emb_dim_ = 4*self.emb_dim
        if len(t.shape) == 3:
            assert (len(h.shape) == 2) & (len(r.shape) == 2)
            r, r_transfer, time_transfer = torch.chunk(r, 3, dim=-1)
            h1 = self.eval_transfer(h, time_transfer, r_transfer)
            # print('t.shape: ',t.shape)
            t1 = self.eval_transfer(t, time_transfer, r_transfer)
            hr = self.eval_calc(h1, r)
            hr = hr.view(hr.shape[0],1,hr.shape[1])
            # print('t1.shape: ',t1.shape, ' hr.shape: ',hr.shape)
            score = torch.sum(hr * t1, -1)
            return score

        elif len(h.shape) == 3:
            assert (len(t.shape) == 2) & (len(r.shape) == 2)
            # this is the head completion case in link prediction
            r, r_transfer, time_transfer = torch.chunk(r, 3, dim=-1)
            # this is the head completion case in link prediction
            h1 = self.eval_transfer(h, time_transfer, r_transfer)
            t1 = self.eval_transfer(t, time_transfer, r_transfer)
            hr = self.eval_calc(h1, r)
            # print('t1.shape: ',t1.shape, ' hr.shape: ',hr.shape)
            t1 = t1.view(t1.shape[0],1,t1.shape[1])
            score = torch.sum(hr * t1, -1)
            return score
        elif len(r.shape) == 3:
            assert (len(h.shape) == 2) & (len(t.shape) == 2)
            # this is the relation prediction case
            r, r_transfer, time_transfer = torch.chunk(r, 3, dim=-1)
            h1 = self.eval_transfer(h, time_transfer, r_transfer)
            t1 = self.eval_transfer(t, time_transfer, r_transfer)
            hr = self.eval_calc(h1, r)
            t1 = t1.view(t1.shape[0],1,hr.shape[1])# needs to be addapted
            score = torch.sum(hr * t1, -1)# needs to be addapted

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx,start_time_idx, end_time_idx, entities=True):
        b_size = h_idx.shape[0]

        h = self.ent_emb(h_idx)
        t = self.ent_emb(t_idx)
        r = self.rel_emb(r_idx)
        
        time_transfer = self.time_transfer(start_time_idx)
        r_transfer = self.rel_transfer(r_idx)

        if entities:
            all_ents = torch.tensor(range(self.n_ent)).to(device)
            candidates = self.ent_emb(all_ents)
            candidates = candidates.expand(b_size, self.n_ent, candidates.shape[-1])
        else:
            all_rels = torch.tensor(range(self.n_rel)).to(device)
            candidates = self.rel_emb(all_rels)
            candidates = candidates.expand(b_size, self.n_rel, candidates.shape[-1])
        return h, t, torch.cat([r, r_transfer, time_transfer], dim = -1), candidates    

In [23]:
class TDYCE_Oct(TempBilinearModel):
    def __init__(self, args):
        super().__init__(args.emb_dim, args.n_entities, args.n_relations)
        self.emb_dim = args.emb_dim
        self.tem_total = args.tem_total
        self.quat_emb_dim = self.emb_dim * 8
        self.ent_emb = nn.Embedding(self.n_ent, self.quat_emb_dim)
        self.rel_emb = nn.Embedding(self.n_rel, self.quat_emb_dim)
        self.rel_transfer = nn.Embedding(self.n_rel, self.quat_emb_dim)
        self.time_transfer = nn.Embedding(self.tem_total, self.quat_emb_dim)
        self.init_weights()
        
    def init_weights(self):
        r, i, j, k, l, m, n, o = oct_init(self.n_ent, self.emb_dim)
        r, i, j, k = torch.from_numpy(r), torch.from_numpy(i), torch.from_numpy(j), torch.from_numpy(k)
        l, m, n, o = torch.from_numpy(l), torch.from_numpy(m), torch.from_numpy(n), torch.from_numpy(o)
        vec1 = torch.cat([r, i, j, k, l, m, n, o], dim=1)
        self.ent_emb.weight.data = vec1.type_as(self.ent_emb.weight.data)

        s, x, y, z, l, m, n, o = oct_init(self.n_rel, self.emb_dim)
        s, x, y, z = torch.from_numpy(s), torch.from_numpy(x), torch.from_numpy(y), torch.from_numpy(z)
        l, m, n, o = torch.from_numpy(l), torch.from_numpy(m), torch.from_numpy(n), torch.from_numpy(o)
        vec2 = torch.cat([s, x, y, z, l, m, n, o], dim=1)
        self.rel_emb.weight.data = vec2.type_as(self.rel_emb.weight.data)
        
        r, i, j, k, l, m, n, o = oct_init(self.tem_total, self.emb_dim)
        r, i, j, k = torch.from_numpy(r), torch.from_numpy(i), torch.from_numpy(j), torch.from_numpy(k)
        l, m, n, o = torch.from_numpy(l), torch.from_numpy(m), torch.from_numpy(n), torch.from_numpy(o)
        vec1 = torch.cat([r, i, j, k, l, m, n, o], dim=1)
        self.time_transfer.weight.data = vec1.type_as(self.time_transfer.weight.data)

        s, x, y, z, l, m, n, o = oct_init(self.n_rel, self.emb_dim)
        s, x, y, z = torch.from_numpy(s), torch.from_numpy(x), torch.from_numpy(y), torch.from_numpy(z)
        l, m, n, o = torch.from_numpy(l), torch.from_numpy(m), torch.from_numpy(n), torch.from_numpy(o)
        vec2 = torch.cat([s, x, y, z, l, m, n, o], dim=1)
        self.rel_transfer.weight.data = vec2.type_as(self.rel_transfer.weight.data)

    def _calc(self, h, r):
        return _omult(h, _onorm(r))

    def eval_calc(self, h, r):
        r = _onorm(r)
        if (len(h.shape) == 3) & (len(r.shape) == 2):
            r = r.view(r.shape[0], 1, r.shape[1])
        elif (len(r.shape) == 3) & (len(h.shape) == 2):
            r = r.view(r.shape[0], 1, r.shape[1])
        return _omult(h, r)

    def _transfer(self, x, x_transfer, r_transfer):
        ent_transfer = self._calc(x, x_transfer)
        ent_rel_transfer = self._calc(ent_transfer, r_transfer)

        return ent_rel_transfer

    def eval_transfer(self, x, x_transfer, r_transfer):
        ent_transfer = self.eval_calc(x, x_transfer)
        # print('ent_transfer.shape: ',ent_transfer.shape)
        # print('r_transfer.shape: ',r_transfer.shape)
        ent_rel_transfer = self.eval_calc(ent_transfer, r_transfer)

        return ent_rel_transfer
    
    
    def scoring_function(self, h_idx, t_idx, r_idx, start_time_idx, end_time_idx):
        h = self.ent_emb(h_idx)
        r = self.rel_emb(r_idx)
        t = self.ent_emb(t_idx)

        # (h, r, t) transfer vector
        time_transfer = self.time_transfer(start_time_idx)
        r_transfer = self.rel_transfer(r_idx)

        h1 = self._transfer(h, time_transfer, r_transfer)
        t1 = self._transfer(t, time_transfer, r_transfer)
        # multiplication as QuatE
        hr = self._calc(h1, r)
        # Inner product as QuatE
        score = quat_dot_prod(hr, t1)
        return score

    def get_embeddings(self):
        self.normalize_parameters()
        return self.ent_emb.weight.data, self.rel_emb.weight.data

    def inference_scoring_function(self, h, t, r, start_time, end_time):
        b_size = h.shape[0]
        emb_dim_ = 4*self.emb_dim
        if len(t.shape) == 3:
            assert (len(h.shape) == 2) & (len(r.shape) == 2)
            r, r_transfer, time_transfer = torch.chunk(r, 3, dim=-1)
            h1 = self.eval_transfer(h, time_transfer, r_transfer)
            # print('t.shape: ',t.shape)
            t1 = self.eval_transfer(t, time_transfer, r_transfer)
            hr = self.eval_calc(h1, r)
            hr = hr.view(hr.shape[0],1,hr.shape[1])
            # print('t1.shape: ',t1.shape, ' hr.shape: ',hr.shape)
            score = torch.sum(hr * t1, -1)
            return score

        elif len(h.shape) == 3:
            assert (len(t.shape) == 2) & (len(r.shape) == 2)
            # this is the head completion case in link prediction
            r, r_transfer, time_transfer = torch.chunk(r, 3, dim=-1)
            # this is the head completion case in link prediction
            h1 = self.eval_transfer(h, time_transfer, r_transfer)
            t1 = self.eval_transfer(t, time_transfer, r_transfer)
            hr = self.eval_calc(h1, r)
            # print('t1.shape: ',t1.shape, ' hr.shape: ',hr.shape)
            t1 = t1.view(t1.shape[0],1,t1.shape[1])
            score = torch.sum(hr * t1, -1)
            return score
        elif len(r.shape) == 3:
            assert (len(h.shape) == 2) & (len(t.shape) == 2)
            # this is the relation prediction case
            r, r_transfer, time_transfer = torch.chunk(r, 3, dim=-1)
            h1 = self.eval_transfer(h, time_transfer, r_transfer)
            t1 = self.eval_transfer(t, time_transfer, r_transfer)
            hr = self.eval_calc(h1, r)
            t1 = t1.view(t1.shape[0],1,hr.shape[1])# needs to be addapted
            score = torch.sum(hr * t1, -1)# needs to be addapted

    def inference_prepare_candidates(self, h_idx, t_idx, r_idx,start_time_idx, end_time_idx, entities=True):
        b_size = h_idx.shape[0]

        h = self.ent_emb(h_idx)
        t = self.ent_emb(t_idx)
        r = self.rel_emb(r_idx)
        
        time_transfer = self.time_transfer(start_time_idx)
        r_transfer = self.rel_transfer(r_idx)

        if entities:
            all_ents = torch.tensor(range(self.n_ent)).to(device)
            candidates = self.ent_emb(all_ents)
            candidates = candidates.expand(b_size, self.n_ent, candidates.shape[-1])
        else:
            all_rels = torch.tensor(range(self.n_rel)).to(device)
            candidates = self.rel_emb(all_rels)
            candidates = candidates.expand(b_size, self.n_rel, candidates.shape[-1])
        return h, t, torch.cat([r, r_transfer, time_transfer], dim = -1), candidates    

In [24]:
def get_model(model_name, args):
    if model_name == 'TDYCE_Quat':
        return TDYCE_Quat(args)
    elif model_name == 'TDYCE_Oct':
        return TDYCE_Oct(args)

## Train Loop

In [25]:
class temporal_train_loop():
    def __init__(self, kg_train, kg_val, kg_test, args):
        self.args = args

        self.kg_train = kg_train
        self.kg_val = kg_val
        self.kg_test = kg_test
        self.n_epochs = args.n_epochs
        self.model_name = args.model_name
        self.emb_dim = args.emb_dim
        self.lr = args.lr
        self.n_epochs = args.n_epochs
        self.b_size = args.b_size
        self.margin = args.margin
        self.model_save_name = args.model_save_name
        self.device = device
        self.model_path = args.model_path
        self.n_neg = args.n_neg

#         self.model = TransEModel(self.emb_dim, self.kg_train.n_ent, self.kg_train.n_rel, dissimilarity_type='L2')
        self.model = get_model(args.model_name, args)
     
        self.criterion = get_loss(args.loss_name, args)

        # Move everything to CUDA if available
        if cuda.is_available():
            cuda.empty_cache()
            self.model = self.model.to(self.device)
            if hasattr(self.criterion, 'to'):
                self.criterion = self.criterion.to(self.device)

        # Define the torch optimizer to be used
        self.optimizer = Adam(self.model.parameters(), lr=self.lr, weight_decay=1e-5)
        # self.sampler = BernoulliNegativeSampler(self.kg_train, n_neg = self.n_neg)
        self.sampler = UniformNegativeSampler(self.kg_train, n_neg = self.n_neg)
        self.dataloader = TempDataLoader(self.kg_train, batch_size=self.b_size, use_cuda=None)

    def save_model(self,):
        torch.save(self.model.state_dict(), join(self.model_path,self.model_save_name))

    def evaluation(self, ):
        with torch.no_grad():
            evaluator = TemporalLinkPredictionEvaluator(self.model, self.kg_val)
            evaluator.evaluate(b_size=100, verbose=False)
            val_res = evaluator.get_results()
            val_filt_mrr = val_res['filt_mrr']
            # print(val_filt_mrr)
            return val_filt_mrr, val_res
    
    def test(self,):
        with torch.no_grad():
            self.model = self.model.to('cpu')
            del self.model
            torch.cuda.empty_cache()
            self.model = get_model(model_name = self.model_name, args = self.args)
            self.model.load_state_dict(torch.load(join(self.model_path,self.model_save_name)))
            os.remove(join(self.model_path,self.model_save_name))
            self.model = self.model.to(device=device)
            evaluator = TemporalLinkPredictionEvaluator(self.model, self.kg_test)
            evaluator.evaluate(b_size=32)
            test_res = evaluator.get_results()
            return test_res

    def run(self):
        
        iterator = tqdm(range(self.n_epochs), unit='epoch')
        eval_epoch = 1
        best_val_mrr = -100
        best_epoch = 0
        patience_max = 9
        patience_counter = 0
        train_evolution = []

        for epoch in iterator:
            running_loss = 0.0
            epoch_log = {}
            for i, batch in enumerate(self.dataloader):
                h, t, r, start_time, end_time = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device), batch[4].to(device)
                # print('h.shape ooooo ',h.shape)
                n_h, n_t = self.sampler.corrupt_batch(h, t, r)                
                self.optimizer.zero_grad()

                pos, neg = self.model(h, t, r, start_time, end_time, n_h, n_t)
                loss = self.criterion(pos, neg)
#                 loss = self.criterion(pos, neg) + self.model.get_reg_loss(h, t, r, start_time, end_time)
                
                if hasattr(self.args, 'add_sin_constaint'):
                    if self.args.add_sin_constaint:
                        loss =loss + 20*self.model.get_constraints(h, start_time, end_time)
                        loss =loss + 20*self.model.get_constraints(t, start_time, end_time)
                
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()
            epoch_log['avg_train_loss'] = running_loss / len(self.dataloader)
            if epoch % eval_epoch == 0:
                val_mrr,  val_res = self.evaluation()
                epoch_log['val_res'] = val_res
                if (best_val_mrr < val_mrr) & (val_mrr<float('inf')):
                    best_val_mrr = val_mrr
                    best_epoch = epoch + 1
                    # save the model
                    self.save_model()
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter == patience_max:
                        break
            train_evolution.append(epoch_log)
            iterator.set_description(
                'Epoch {} | mean loss: {:.5f}| best mrr: {:.5f} | Best Epoch {} '.format(epoch + 1,
                                                      running_loss / len(self.dataloader), best_val_mrr, best_epoch))

        # self.model.normalize_parameters()
        return train_evolution


## Training

In [26]:
res_save_file = 'TDYCE_experiments_log.jsonl'
if os.path.exists(res_save_file):
    all_exp_df = check_experiment(res_save_file)
else:
    all_exp_df = None
args = Params()

args.n_epochs = 1000
# args.margin = 200
args.model_save_name = 'best_de_model_.bt'
args.loss_name = 'MarginLoss'
# args.loss_name = 'BinaryCrossEntropyLoss'
args.model_path = 'best_models'
args.time_mode = 'simple'
# args.n_neg = 10
# args.flag_hamilton_mul_norm = False
# args.input_dropout = .3
# args.hidden_dropout = .3
# 0.36
# args.time_proj_method = 'tero'
def get_model_name(dataset,batch_size , lr , emb_dim , model_name , dissimilarity_type):
    vals = [dataset,batch_size , lr , emb_dim , model_name , dissimilarity_type]
    keys = ['dataset','batch_size' , 'lr' , 'emb_dim' , 'model_name' , 'dissimilarity_type']
    kv = []
    for k,  v in zip(keys,vals):
        kv.append(':'.join([str(k),  str(v)]))
    kv = '$'.join(kv)
    kv = kv+'.pt'
    return kv

args.time_proj_method = 'ent'#ent_rel 
for model_name in ['TDYCE_Quat']: # TDYCE_Oct TDYCE_Quat
    args.model_name = model_name
    for dataset in ['gdelt']:# icews14 gdelt wikidata12k icews15
        args.dataset = dataset
        kg_train, kg_val, kg_test = get_data(args.dataset, args.time_mode)
        args.n_entities = kg_train.n_ent
        args.n_relations = kg_train.n_rel
        args.tem_total = kg_train.n_time
        for n_neg in [5,2,10,1]:
            args.n_neg = n_neg
            for margin in [200,100,10]:
                args.margin = margin
                for b_size in [1000, 500]:
                    # 500, 1000, 2000
                    args.b_size = b_size
                    for lr in [0.001, 0.01]:
                        args.lr = lr
                        for emb_dim in [100]:
                            args.emb_dim = emb_dim
                            args.num_features = args.emb_dim

                            if args.model_name in ['TeRoTransEModel']:
                                dissimilarity_type_list = ['L1']
                                # 'L2', 'L1'
                            elif args.model_name in ['toruse']:
                                dissimilarity_type_list = ['L1', 'torus_L1', 'torus_L2', 'torus_eL2']
                            else:
                                dissimilarity_type_list = ['']

                            for dissimilarity_type in dissimilarity_type_list:
                                if all_exp_df is not None:
                                    exp_already_done = experiment_exists(all_exp_data_df = all_exp_df, exp_data_dict = args.__dict__)
                                else:
                                    exp_already_done = False
                                model_save_name = get_model_name(dataset, args.b_size , lr , emb_dim , args.model_name , dissimilarity_type)
                                print(exp_already_done)
                                print(model_save_name)
#                                 exp_already_done = False
                                if not exp_already_done:
                                    args.dissimilarity_type = dissimilarity_type
                                    tmp_trainlp = temporal_train_loop(kg_train, kg_val, kg_test, args)
                                    train_evolution = tmp_trainlp.run()
                                    test_res = tmp_trainlp.test()
                                    train_log = {}
                                    print(test_res)
                                    train_log['train_params'] = args.__dict__
                                    train_log['train_evolution'] = train_evolution
                                    train_log['test_results'] = test_res
                                    write_json_lines(file_name =res_save_file, dict_data = train_log)


True
dataset:gdelt$batch_size:1000$lr:0.001$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:1000$lr:0.01$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:500$lr:0.001$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:500$lr:0.01$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:1000$lr:0.001$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:1000$lr:0.01$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:500$lr:0.001$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:500$lr:0.01$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:1000$lr:0.001$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt
True
dataset:gdelt$batch_size:1000$lr:0.01$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt

  0%|          | 0/1000 [00:00<?, ?epoch/s]

Link prediction evaluation:   0%|          | 0/10687 [00:00<?, ?batch/s]

{'hit_1': 0.07013372704386711, 'filt_hit_1': 0.08510619401931763, 'hit_2': 0.1282324567437172, 'filt_hit_2': 0.1480812132358551, 'hit_3': 0.1694842427968979, 'filt_hit_3': 0.18957863748073578, 'hit_4': 0.20179640501737595, 'filt_hit_4': 0.22137758880853653, 'hit_5': 0.2287994772195816, 'filt_hit_5': 0.24795664101839066, 'hit_6': 0.2524571567773819, 'filt_hit_6': 0.2709270864725113, 'hit_7': 0.2733805924654007, 'filt_hit_7': 0.2911267578601837, 'hit_8': 0.2920347601175308, 'filt_hit_8': 0.3092428594827652, 'hit_9': 0.309162437915802, 'filt_hit_9': 0.32582223415374756, 'hit_10': 0.3247402459383011, 'filt_hit_10': 0.34128306806087494, 'mr': 67.00736236572266, 'filt_mr': 65.19158935546875, 'head_mrr': 0.15534041821956635, 'tail_mrr': 0.15721067786216736, 'filt_head_mrr': 0.17211297154426575, 'filt_tail_mrr': 0.17342403531074524, 'mrr': 0.15627554059028625, 'filt_mrr': 0.1727685034275055}
False
dataset:gdelt$batch_size:500$lr:0.01$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt


  0%|          | 0/1000 [00:00<?, ?epoch/s]

...

Link prediction evaluation:   0%|          | 0/10687 [00:00<?, ?batch/s]

{'hit_1': 0.07171870395541191, 'filt_hit_1': 0.08642798662185669, 'hit_2': 0.12860091775655746, 'filt_hit_2': 0.14817771315574646, 'hit_3': 0.16941697895526886, 'filt_hit_3': 0.18915022164583206, 'hit_4': 0.20126856863498688, 'filt_hit_4': 0.2209111526608467, 'hit_5': 0.22813566029071808, 'filt_hit_5': 0.24720655381679535, 'hit_6': 0.2518152743577957, 'filt_hit_6': 0.2700074017047882, 'hit_7': 0.27241410315036774, 'filt_hit_7': 0.2900681644678116, 'hit_8': 0.2909673899412155, 'filt_hit_8': 0.3082837015390396, 'hit_9': 0.3079225420951843, 'filt_hit_9': 0.3248031288385391, 'hit_10': 0.32352666556835175, 'filt_hit_10': 0.3399510383605957, 'mr': 67.87479400634766, 'filt_mr': 66.06599426269531, 'head_mrr': 0.15553192794322968, 'tail_mrr': 0.15803104639053345, 'filt_head_mrr': 0.1719973236322403, 'filt_tail_mrr': 0.1740785837173462, 'mrr': 0.15678149461746216, 'filt_mrr': 0.17303794622421265}
False
dataset:gdelt$batch_size:500$lr:0.01$emb_dim:100$model_name:TDYCE_Quat$dissimilarity_type:.pt


  0%|          | 0/1000 [00:00<?, ?epoch/s]

Link prediction evaluation:   0%|          | 0/10687 [00:00<?, ?batch/s]

{'hit_1': 0.05980798974633217, 'filt_hit_1': 0.07190147414803505, 'hit_2': 0.11098195612430573, 'filt_hit_2': 0.1264413222670555, 'hit_3': 0.14790721237659454, 'filt_hit_3': 0.16406110674142838, 'hit_4': 0.17832297086715698, 'filt_hit_4': 0.19465523958206177, 'hit_5': 0.20444583147764206, 'filt_hit_5': 0.22060702741146088, 'hit_6': 0.22683434188365936, 'filt_hit_6': 0.2425057217478752, 'hit_7': 0.24668894708156586, 'filt_hit_7': 0.2619713842868805, 'hit_8': 0.2646617591381073, 'filt_hit_8': 0.2794061303138733, 'hit_9': 0.28096917271614075, 'filt_hit_9': 0.29530707001686096, 'hit_10': 0.29586707055568695, 'filt_hit_10': 0.31009677052497864, 'mr': 72.37879943847656, 'filt_mr': 70.55870056152344, 'head_mrr': 0.13890808820724487, 'tail_mrr': 0.14091819524765015, 'filt_head_mrr': 0.15260377526283264, 'filt_tail_mrr': 0.1541258841753006, 'mrr': 0.1399131417274475, 'filt_mrr': 0.15336483716964722}
